In [80]:
import pandas as pd
from time import time
from glob import glob

In [2]:
cons = pd.read_csv("../data/external/cons46way/phastConsElements46wayPrimates.csv")

In [70]:
cons[cons.chrom == "chr1"].shape

(66907, 6)

In [56]:
dataset = pd.read_csv("../data/processed/humsavar_protparam_gt.tab.gz", sep="\t", usecols=["MUTANT"]).squeeze() \
            .str.split("-").apply(lambda x: x[0])

In [61]:
hum_clean = pd.read_csv("../data/interim/humsavar_clean_201711.csv.gz")
hum_clean = hum_clean[hum_clean["Swiss Prot AC"].isin(dataset)]
hum_clean = hum_clean[~hum_clean.dbSNP.isnull()]
hum_clean = hum_clean[["dbSNP"]]

In [62]:
dbsnp = pd.read_csv("../data/external/cons46way/dbsnp.csv.gz")

In [63]:
hum_clean = hum_clean.merge(dbsnp, right_on="name", left_on="dbSNP", how="left")

In [67]:
%xdel dbsnp

In [64]:
hum_clean = hum_clean[(hum_clean.chromEnd - hum_clean.chromStart == 1)]

In [66]:
hum_clean = hum_clean[["dbSNP", "chrom", "chromStart", "chromEnd"]]

In [79]:
for chrom in hum_clean.chrom.unique():
    timeInit = time()
    print("Chromosome: {}".format(chrom))
    cons_filtered = cons[cons.chrom == chrom]
    hum_clean_filtered = hum_clean[hum_clean.chrom == chrom]
    merging_chrom = cons_filtered.merge(hum_clean_filtered, on="chrom")
    merging_chrom = merging_chrom[(merging_chrom.chromStart_y >= merging_chrom.chromStart_x) &
                                 (merging_chrom.chromEnd_y <= merging_chrom.chromEnd_x)]
    merging_chrom.to_csv("../data/interim/chrom/{}.csv".format(chrom), index=False)
    timeEnd = time()
    print("Time elapsed: {}s".format(timeEnd - timeInit))

Chromosome: chr2
Time elapsed: 50.1989004611969s
Chromosome: chr11
Time elapsed: 3.3955438137054443s
Chromosome: chr16
Time elapsed: 1.0275089740753174s
Chromosome: chr1
Time elapsed: 15.910726070404053s
Chromosome: chr18
Time elapsed: 0.9041519165039062s
Chromosome: chr10
Time elapsed: 1.3800997734069824s
Chromosome: chr17
Time elapsed: 1.269038438796997s
Chromosome: chr9
Time elapsed: 1.5352327823638916s
Chromosome: chr5
Time elapsed: 2.4422969818115234s
Chromosome: chr14
Time elapsed: 1.4965333938598633s
Chromosome: chr7
Time elapsed: 2.688612461090088s
Chromosome: chr3
Time elapsed: 6.544106483459473s
Chromosome: chr4
Time elapsed: 1.4155011177062988s
Chromosome: chr20
Time elapsed: 0.30748939514160156s
Chromosome: chrX
Time elapsed: 1.3936080932617188s
Chromosome: chr6
Time elapsed: 1.7238636016845703s
Chromosome: chr19
Time elapsed: 0.6064441204071045s
Chromosome: chr15
Time elapsed: 1.1361281871795654s
Chromosome: chr8
Time elapsed: 1.2732443809509277s
Chromosome: chr22
Time ela

In [83]:
res = []
for f in glob("../data/interim/chrom/*"):
    df = pd.read_csv(f)
    res.append(df)
res = pd.concat(res)

In [90]:
res = res[["dbSNP", "score"]]

In [91]:
res.head()

,dbSNP,score
0,rs35609073,389.0
1,rs34953021,342.0
2,rs774996232,347.0
3,rs758268804,347.0
4,rs34872409,337.0


In [97]:
for e, group in enumerate(res[res.dbSNP.duplicated(keep=False)].groupby("dbSNP")["score"]):
    if group[1].value_counts().shape[0] != 1:
        print("ERROR!")

In [98]:
res.drop_duplicates(inplace=True)

In [100]:
res.shape

(5365, 2)

In [101]:
res.to_csv("../data/interim/cons46way.csv", index=False)